In [1]:
import pandas as pd
import re
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import column_or_1d
import numpy as np
import pickle
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from xgboost import XGBClassifier

In [2]:
!pip install catboost

In [3]:
import catboost
from catboost import cv, Pool, CatBoostClassifier

# Dữ liệu

In [4]:
df_train_info = pd.read_csv('/content/info_train.csv') 
df_train_work = pd.read_csv('/content/work_train.csv')
df_train_label = pd.read_csv('/content/label_train.csv') 
df_test_info = pd.read_csv('/content/info_test.csv')  
df_test_work = pd.read_csv('/content/work_test.csv')
df_test_label = pd.read_csv('/content/label_test.csv')

In [5]:
fts = pd.merge(left=df_train_work, right=df_train_info, how='left', on='id_bh')
df = pd.merge(left=fts, right=df_train_label, how='left', on='id_bh')

In [6]:
df['address'] = df['address_x'].combine_first(df['address_y'])

In [7]:
df.head()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,address_x,bithYear,gender,address_y,label,address
0,1,113039360,106,TF2212F,-1,Giám đốc,20130100,20151200,7.0,Hà Nội,1971,MALE,hà Nội,4,Hà Nội
1,1,113039360,106,TF2212F,-1,Giám đốc,20160100,20220400,10.0,Hà Nội,1971,MALE,hà Nội,4,Hà Nội
2,2,116074930,102,TB16010,-1,Nhân viên lễ tân,20160600,20161200,7.0,Hà Nội,1993,FEMALE,Thành phố Hà Nội,2,Hà Nội
3,2,116074930,102,TB16010,-1,Nhân viên lễ tân,20170100,20170300,8.0,Hà Nội,1993,FEMALE,Thành phố Hà Nội,2,Hà Nội
4,2,116074930,102,NaN,-1,NaN,20170400,20170700,-1.0,NaN,1993,FEMALE,Thành phố Hà Nội,2,Thành phố Hà Nội


In [8]:
df_filter = df.sort_values(
    by=['id_bh', 'to_date', 'from_date'], 
    ascending=[True, False, False]
).drop_duplicates(
    subset=['id_bh'], 
    keep='first', 
    ignore_index=True
)

In [9]:
def split_year(string):
    return string[:4]

def split_month(string):
    return string[4:6]

def split_month_year(string):
    return str(string[:6])

In [10]:
df_filter['from_date_new']= df_filter['from_date'].astype('str').apply(split_month_year)
df_filter['end_date_new']= df_filter['to_date'].astype('str').apply(split_month_year)

In [11]:
df_filter['exp'] = pd.to_datetime(df_filter['end_date_new'].apply(str), format='%Y%m') - pd.to_datetime(df_filter['from_date_new'].apply(str), format='%Y%m')
df_filter['exp'] = df_filter['exp'].dt.days

In [12]:
df_filter['age'] = 2022 - df_filter['bithYear'].astype(int)

In [13]:
df_filter.head()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,address_x,bithYear,gender,address_y,label,address,from_date_new,end_date_new,exp,age
0,4686,100000725,100,HW01180,6,"CVC, TP",20210600,20220400,17.0,HN,1963,MALE,Hà Nội,5,HN,202106,202204,304,59
1,29905,100007067,2400,TA0002A,1,Nhân viên bán xăng dầu,20190100,20220400,9.0,TP Bắc Giang,1971,MALE,Bắc Giang,4,TP Bắc Giang,201901,202204,1186,51
2,4503,100007555,109,TI3361I,-1,Nhân viên,20211000,20220400,9.0,Việt Nam,1970,MALE,NaN,2,Việt Nam,202110,202204,182,52
3,2939,100008102,100,HW0013Z,-1,"Chuyên viên chính, Ủy viên Thường trực Ủy ban...",20200800,20220400,18.0,Hà Nội,1970,MALE,HN,5,Hà Nội,202008,202204,608,52
4,8967,100008777,114,HN05360,-1,PCT UBND huyện,20201000,20220400,17.0,HÀ NỘI,1965,MALE,NaN,6,HÀ NỘI,202010,202204,547,57


In [14]:
fts_name = ['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'exp']
cat_fts_idx = [0, 1, 2, 4]

In [15]:
df_filter['id_office'].fillna('unknow', inplace=True)

#Mô hình

In [16]:
cv_data = Pool(data=df_filter[fts_name], label=df_filter['label'], cat_features=cat_fts_idx)

In [17]:
params = {"iterations": 500,
          'random_seed': 42,
          'bagging_temperature': 0.1,
          'l2_leaf_reg': 10,
          'leaf_estimation_iterations': 5,
          "loss_function": "MultiClass", 
          'custom_metric': ['TotalF1:average=Macro', 'TotalF1:average=Weighted'] 
        }

scores, models = cv(cv_data, params, fold_count=5, return_models=True)

Training on fold [0/5]
0:	learn: 1.8337946	test: 1.8334022	best: 1.8334022 (0)	total: 265ms	remaining: 2m 12s
1:	learn: 1.7218713	test: 1.7213940	best: 1.7213940 (1)	total: 525ms	remaining: 2m 10s
2:	learn: 1.6162330	test: 1.6156326	best: 1.6156326 (2)	total: 766ms	remaining: 2m 6s
3:	learn: 1.5215663	test: 1.5208442	best: 1.5208442 (3)	total: 1.01s	remaining: 2m 5s
4:	learn: 1.4296854	test: 1.4288252	best: 1.4288252 (4)	total: 1.25s	remaining: 2m 3s
5:	learn: 1.3451346	test: 1.3440522	best: 1.3440522 (5)	total: 1.47s	remaining: 2m
6:	learn: 1.2652053	test: 1.2641641	best: 1.2641641 (6)	total: 1.7s	remaining: 2m
7:	learn: 1.1964743	test: 1.1954502	best: 1.1954502 (7)	total: 1.94s	remaining: 1m 59s
8:	learn: 1.1346984	test: 1.1337569	best: 1.1337569 (8)	total: 2.16s	remaining: 1m 57s
9:	learn: 1.0719222	test: 1.0710928	best: 1.0710928 (9)	total: 2.39s	remaining: 1m 57s
10:	learn: 1.0206080	test: 1.0197090	best: 1.0197090 (10)	total: 2.63s	remaining: 1m 57s
11:	learn: 0.9699615	test: 0.9

In [19]:
df_test = pd.merge(left=df_test_work, right=df_test_info, how='left', on='id_bh')

In [20]:
df_test = df_test.sort_values(
    by=['id_bh', 'to_date', 'from_date'], 
    ascending=[True, False, False]
).drop_duplicates(
    subset=['id_bh'], 
    keep='first', 
    ignore_index=True
)

In [24]:
df_test['from_date_new']= df_test['from_date'].astype('str').apply(split_month_year)
df_test['end_date_new']= df_test['to_date'].astype('str').apply(split_month_year)

In [25]:
df_test['exp'] = pd.to_datetime(df_test['end_date_new'].apply(str), format='%Y%m') - pd.to_datetime(df_test['from_date_new'].apply(str), format='%Y%m')
df_test['exp'] = df_test['exp'].dt.days

In [22]:
df_test['age'] = 2022 - df_test['bithYear'].astype(int)

In [26]:
df_test.head()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,address_x,bithYear,gender,address_y,age,from_date_new,end_date_new,exp
0,3895,100000688,105,TE1931E,-1,Lái máy,20190100,20220400,9.0,Hà Nội,1973,MALE,NaN,49,201901,202204,1186
1,3131,100005503,107,TG14845,-1,Nhân viên quản lý,20200100,20220400,9.0,HN,1971,FEMALE,NaN,51,202001,202204,821
2,2828,100006307,105,HW05886,6,NaN,20120300,20220400,16.0,NaN,1952,MALE,NaN,70,201203,202204,3683
3,3865,100008625,106,QW02899,-1,NaN,20180100,20220400,15.0,NaN,1973,MALE,Hà Nội,49,201801,202204,1551
4,3841,100009238,4015,TO08665,-1,Lao động phổ thông,20200100,20211200,6.0,Tỉnh Nghệ An,1978,MALE,Nghệ an,44,202001,202112,700


In [27]:
pred_probs_1 = models[0].predict(df_test[fts_name], prediction_type='Probability')
pred_probs_2 = models[1].predict(df_test[fts_name], prediction_type='Probability')
pred_probs_3 = models[2].predict(df_test[fts_name], prediction_type='Probability')
pred_probs_4 = models[3].predict(df_test[fts_name], prediction_type='Probability')
pred_probs_5 = models[4].predict(df_test[fts_name], prediction_type='Probability')

In [28]:
pred_probs = (pred_probs_1+pred_probs_2+pred_probs_3+pred_probs_4+pred_probs_5)/5

In [29]:
df_test['label'] = (pred_probs.argmax(axis=1)+1)

In [30]:
res = df_test[['id_bh', 'label']]

In [31]:
submit_df = pd.merge(df_test_label, res, 'inner', 'id_bh')

In [32]:
submit_df.to_csv("/content/submission.csv", index=False)